<center><h1>Dataset Selection(Part 2)</h1></center>
***

In the previous notebook you saw that imputed dataset was better on crossvalidation, now since I have two diferent datasets, I want to check which of them will perform better on completely new dataset, we will see which dataset is more biased. Let's at first download the test set here. 

In [2]:
import pandas as pd
import warnings
import set_jupyter_path
from src.car_price_prediction.utils import dataset_manager, df_utils
from sklearn.ensemble import RandomForestRegressor
from notebook_helper_functions.data_impute import get_imputed_dataset

In [3]:
data = dataset_manager.get_cleaned_outliers_dataset()

def make_imputed_dataset(data):
    data_for_imp = data.copy()
    #erroneous warnings in imported library
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data_imp = get_imputed_dataset(data_for_imp)
    return data_imp

def get_max_unbiased_dataset():
    return dataset_manager.get_processed_dataset()

imputed_data = make_imputed_dataset(data)
unbiased_data = get_max_unbiased_dataset()

In [4]:
X_imp,y_imp = df_utils.get_data_and_target(imputed_data)
X_unbias,y_unbias = df_utils.get_data_and_target(unbiased_data)

Here, I will load new dataset for testing, and make same dummy variables between train and test.

In [5]:
test_data = pd.read_excel("../data/external/test_data.xlsx")
X_test,y_test = df_utils.get_data_and_target(test_data)
X_imp,X_test_imp = df_utils.make_dummies(X_imp,X_test)
X_unbias,X_test_unbias = df_utils.make_dummies(X_unbias, X_test)

I will compare the datasets by RandomForest.

In [6]:
forest_imp = RandomForestRegressor(n_estimators=30)
forest_unbias = RandomForestRegressor(n_estimators=30)

forest_imp.fit(X_imp,y_imp)
forest_unbias.fit(X_unbias,y_unbias)

y_pred_imp = forest_imp.predict(X_test_imp)
y_pred_unbias = forest_unbias.predict(X_test_unbias)

And finally we can clearly see, which dataset is least biased. This error is due to bias in imputting dataset.

In [7]:
abs(y_pred_imp - y_test).sum()/len(y_test)

3820.5041530054646

In [8]:
abs(y_pred_unbias-y_test).sum()/len(y_test)

1711.8107201405153

By this notebook, I finished preprocessing part of making prediction algorithm.